In [9]:
#load path
import os
import sys
import astropy
from astropy.io import fits
from astropy.table import QTable

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, sigma_clip

from astropy.wcs import WCS
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus, ApertureStats
from astropy.stats import SigmaClip

from astropy.visualization import (ImageNormalize, MinMaxInterval,PercentileInterval,SqrtStretch)
import glob

import warnings


In [10]:
path='./data/'
rawpath=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
calibpath='./data/Calibs/'
reduxpath='./data/Redux/'
combinepath='./data/Redux/Combined/'
gain=0.25 #e-/ADU at G = 125

In [11]:
def GAIA_query(center_ra, center_dec, radius, magmin=10, magmax=14, writedir='./calstars/'):
   from astroquery.gaia import GaiaClass
   from astropy.coordinates import SkyCoord
   import astropy.units as u


   coord = SkyCoord(ra=center_ra, dec=center_dec, unit=(u.deg, u.deg))
   gaiaobj = GaiaClass()
   gaiaobj.ROW_LIMIT = 50000  # High limit on number of rows returned
   job = gaiaobj.cone_search(coord, radius=radius * u.deg, verbose=False)
   results = job.get_results()

   selected = (results['has_xp_sampled']) & (results['phot_g_mean_mag'] < magmax) & (results['phot_g_mean_mag'] > magmin)
   starID = results['source_id'][selected]
   starRA = results['ra'][selected]
   starDEC = results['dec'][selected]

   print('Found {} stars'.format(len(starID)))
   ## Download the spectra:
   retrieval_type = 'XP_SAMPLED'
   datalink = gaiaobj.load_data(ids=starID,
                          data_release='Gaia DR3',
                          retrieval_type=retrieval_type,
                          data_structure='INDIVIDUAL',
                          format='votable',
                          verbose=False,
                          overwrite_output_file='True')

   dl_keys  = [inp for inp in datalink.keys()]

   strwave = []
   strflux = []

   #flux is in w/m2/nm, convert to erg/s/cm2/A
   conv_factor = 1e7 * 1e-4 * 1e-1 #1 W = 1e7 erg/s, 1 m2 = 1e4 cm2, 1 nm = 10 A

   for ind, strid in enumerate(starID):
         for dl_key in dl_keys:
              if str(strid) in dl_key:
                wave = datalink[dl_key][0].to_table().as_array()['wavelength']
                wave = np.ma.getdata(wave)*10
                flux = datalink[dl_key][0].to_table().as_array()['flux']
                flux = np.ma.getdata(flux)*conv_factor
                strwave.append(wave)
                strflux.append(flux)
         #Save in fits file
         hdu1 = fits.PrimaryHDU(flux)
         hdu1.header['SRCID'] = starID[ind]
         hdu1.header['RA'] = starRA[ind]
         hdu1.header['DEC'] = starDEC[ind]
         hdu1.header['DEC'] = starDEC[ind]
         hdu1.header['FUNIT'] = 'erg/s/cm2/A'
         hdu1.header['CRVAL1'] = wave[0]
         hdu1.header['CD1_1'] = wave[1]-wave[0]
         hdu2 = fits.ImageHDU(wave)
         hdulist = fits.HDUList([hdu1, hdu2])
         hdulist.writeto(writedir+'GAIA3_{}.fits'.format(starID[ind]), overwrite=True)
    
   return starID, starRA, starDEC, np.array(strwave), np.array(strflux)

In [12]:
def make_star_library(header, img, caldir, force_query=False):

    #Find center of image in RA and DEC
    wcs = WCS(header)
    ny , nx = img.shape
    center_ra, center_dec = wcs.wcs_pix2world(nx/2, ny/2, 0)

    objname = header['OBJECT'].replace(' ','_')
    caltargdir = caldir+objname+'/'
    if not os.path.exists(caltargdir):
       os.makedirs(caltargdir)

    fcalstr = glob.glob(caltargdir+'GAIA3_*.fits')
    if len(fcalstr) == 0 or force_query==True:

      print('Running GAIA query....')  
      #Query GAIA for stars in the field
      radius = 0.4 #degrees
      strID, strRA, strDEC, specwave, specflux = GAIA_query(center_ra, center_dec, radius, \
                                                 magmin=10, magmax=14, writedir=caltargdir)
    else:
      print('Reading GAIA spectra from: {}'.format(caltargdir))
      strID = []
      strRA = []
      strDEC = []
      specwave = []
      specflux = []

      for fcal in fcalstr:
          hdu = fits.open(fcal)
          strID.append(hdu[0].header['SRCID'])
          strRA.append(hdu[0].header['RA'])
          strDEC.append(hdu[0].header['DEC'])
          specwave.append(hdu[1].data)
          specflux.append(hdu[0].data)
          hdu.close()

      specwave = np.array(specwave)
      specflux = np.array(specflux) 
      
    #For each star, find its pixel position in the image
    star_positions = []
    for ra, dec in zip(strRA, strDEC):
        x, y = wcs.wcs_world2pix(ra, dec, 0)
        star_positions.append((x, y))
    star_positions = np.array(star_positions)

    #keep only stars within the image boundaries
    okstars =  (0 < star_positions[:,0]) & (star_positions[:,0] < nx) \
               & (star_positions[:,1] < ny) & (0 < star_positions[:,1])        
    
    star_positions = star_positions[okstars]
    specwave = specwave[okstars,:]
    specflux = specflux[okstars,:]

    return star_positions, specwave, specflux


In [13]:
def flux_cal_gaia(img, mask, header, filter, caldir='./calstars/'):

    star_positions, specwave, specflux = make_star_library(header, img, caldir)
    if len(star_positions)==0:
       star_positions, specwave, specflux = make_star_library(header, img, caldir, force_query=True) 
       if len(star_positions)==0:
           return 0, 0, 0, 2 #Mask value = 2 means no stars found even after forcing query
         
    #Perform aperture photometry on each star with background subtraction
    apertures = CircularAperture(star_positions, r=10)
    annulus = CircularAnnulus(star_positions, r_in=12, r_out=15)

    sigclip = SigmaClip(sigma=3.0, maxiters=10)
    aper_stats = ApertureStats(img, apertures, sigma_clip=None)
    bkg_stats = ApertureStats(img, annulus, sigma_clip=sigclip)
    bkg_sum = bkg_stats.median * aper_stats.sum_aper_area.value
    inst_flux = aper_stats.sum - bkg_sum

    mask_table = aperture_photometry(mask,apertures)
    satpix = mask_table['aperture_sum']
    
    #Keep only stars with positive fluxes and unsaturated
    okstars = (inst_flux>0) & (np.isfinite(inst_flux)) & (satpix==0)
    if okstars.sum()==0:
        print('No valid stars found for flux calibration. All stars are either saturated or have non-positive fluxes.')
        return 0, 0, 0, 4 #Mask value = 4 means no valid stars left after removing saturation and negative fluxes

    star_positions = star_positions[okstars]
    inst_flux = inst_flux[okstars]
    specwave = specwave[okstars,:]
    specflux = specflux[okstars,:]    

    #Read appropriate filter file
    filterdict = {'R':'r', 'G':'g', 'I':'i', 'SII':'SII', 'OIII':'OIII', 'Ha':'Ha', 'Hb':'Hb'}
    filterfile = './filters/Tobi_'+filterdict[filter]+'.txt'
    filterdata = np.loadtxt(filterfile)

    #compute filter pivot wavelength
    filter_wave = filterdata[:,1]
    filter_trans = filterdata[:,2]
    num = np.trapz(filter_wave * filter_trans, filter_wave)
    denom = np.trapz(filter_trans, filter_wave)
    filter_pivot = num / denom

    #convolve each spectrum with the filter
    calfactors = []
    psffwhm = []
    psfaxratio = []
    
    for i in range(okstars.sum()):
        wave = specwave[i]
        flux = specflux[i]
        #interpolate filter to spectrum wavelength
        filter_interp = np.interp(wave, filter_wave, filter_trans, left=0, right=0)
        #compute the integral of flux * filter
        num = np.trapz(flux * filter_interp, wave)
        denom = np.trapz(filter_interp, wave)
        flux_cal = num / denom  # erg/s/cm2/A
        #compute the calibration factor
        calfactor = flux_cal / inst_flux[i]  # erg/s/cm2/A per e-/s
        calfactors.append(calfactor)   

    #return median calibration factor
    calfactors=np.array(calfactors)
    mean_calfactor, median_calfactor, std_calfactor = sigma_clipped_stats(calfactors, sigma=3.5)
    #Compute AB ZP from this calib factor
    median_calfacnu = median_calfactor * (filter_pivot**2) / 2.99792458e18
    median_abzp = -2.5 * np.log10(median_calfacnu) - 48.6

    #Done with fluxcal now do PSF fitting
    from photutils.psf import PSFPhotometry, GaussianPRF
    from photutils.background import LocalBackground, MMMBackground
    localbkg_estimator = LocalBackground(5, 10, MMMBackground())

    for i in range(okstars.sum()):
      psf_model = GaussianPRF()
      psf_model.x_fwhm.fixed = False
      psf_model.y_fwhm.fixed = False
      psf_model.theta.fixed = False
      init_params = QTable()
      init_params['x'] = [star_positions[i,0]]
      init_params['y'] = [star_positions[i,1]]
      init_params['x_fwhm'] = [4]
      init_params['y_fwhm'] = [4]
      psfphot = PSFPhotometry(psf_model, (9,9), aperture_radius=4,localbkg_estimator=localbkg_estimator)
      phot = psfphot(img, init_params=init_params)
      #for p in phot.keys():
      #    print(phot[p])
      psffwhm.append(np.min([phot['x_fwhm_fit'][0],phot['y_fwhm_fit'][0]]))
      psfaxratio.append(np.max([phot['x_fwhm_fit'][0],phot['y_fwhm_fit'][0]])/np.min([phot['x_fwhm_fit'][0],phot['y_fwhm_fit'][0]]))

    print('  Image Calibration factor in filter {}: {:.2e} +/- {:.2e} (AB ZP {:.2f})'.format(filter, median_calfactor, std_calfactor, median_abzp))

    psffwhm=np.array(psffwhm)
    mean_fwhm, median_fwhm, std_fwhm = sigma_clipped_stats(psffwhm, sigma=3.5)

    psfaxratio=np.array(psfaxratio)
    mean_axratio, median_axratio, std_axratio = sigma_clipped_stats(psfaxratio, sigma=3.5)

    print('  PSF FWHM: {:.2f} arcsec. Axis Ratio: {:.2f}'.format(median_fwhm*0.55, median_axratio))


    return [median_calfactor, std_calfactor, median_abzp, len(calfactors), 0], [median_fwhm, std_fwhm, median_axratio, std_axratio]



In [14]:
#now reduce science images
#in every directory of rawpath, find if there is a LIGHT folder and reduce the images inside
diffrun=True
addsuperflat=False
cutsize = 400
for root, dirs, files in os.walk(rawpath):
    if 'LIGHT' in dirs:

        light_dir = os.path.join(root, 'LIGHT')
        sciencelist=glob.glob(light_dir+'/*sci*.fits')
        #sciencelist=glob.glob(light_dir+'/2025-11-07*sci_PSZ*.fits')
        if len(sciencelist)==0:
            continue

        date = root.split('/')[-1]
        print('Reducing science images for date: {}'.format(date))
        #find closest date in calibpath
        allbias = glob.glob(calibpath+'/MasterBias_*.fits')
        alldates = [os.path.basename(f).split('_')[1].split('.')[0] for f in allbias]
        date_diffs = [abs((astropy.time.Time(date) - astropy.time.Time('{}-{}-{}'.format(d[:4],d[4:6],d[6:8]))).jd) for d in alldates]
        closest_date = alldates[np.argmin(date_diffs)]
        print('Using calibration bias from date: {}-{}-{}'.format(closest_date[:4],closest_date[4:6],closest_date[6:8]))

        masterbias=fits.open(calibpath+'/MasterBias_{}.fits'.format(closest_date))[0].data

        for img in sciencelist:
          if diffrun and os.path.isfile(reduxpath+"/"+img.replace('.fits','_reduced.fits').split('/')[-1]):
              print('Skipping file {}'.format(img))
              continue
          else:
           hdu=fits.open(img)
           data=hdu[0].data
           header=hdu[0].header
           filter=header['FILTER']
           exptime=header['EXPTIME']
           #Read also flat and dark (darks are fixed in time so far)

           allflats = glob.glob(calibpath+'/MasterFlat_{}_*.fits'.format(filter))
           alldates = [os.path.basename(f).split('_')[-1].split('.')[0] for f in allflats]
           date_diffs = [abs((astropy.time.Time(date) - astropy.time.Time('{}-{}-{}'.format(d[:4],d[4:6],d[6:8]))).jd) for d in alldates]
           closest_date = alldates[np.argmin(date_diffs)]
           print('  Using calibration flat from date: {}-{}-{}'.format(closest_date[:4],closest_date[4:6],closest_date[6:8]))

           normflat=fits.open(calibpath+'MasterFlat_{}_{}.fits'.format(filter, closest_date))[0].data
           try:
               masterdark = fits.open(calibpath+'MasterDark_'+str(int(exptime))+'s.fits')[0].data
               masterdark -= masterbias
               darkval = np.median(masterdark)
           except:
               masterdark = fits.open(calibpath+'MasterDark_1200s.fits')[0].data
               masterdark -= masterbias
               darkval = np.median(masterdark)/1200.*exptime
               print('  No matching dark found for exptime = {}s, using scaled 1200s dark'.format(exptime))
           try:
               if addsuperflat:
                     dummy = fits.open(calibpath+'SuperFlat_{}.fits'.format(filter))[0].data
                     superflat= np.ones_like(normflat)
                     superflat[cutsize:-cutsize,cutsize:-cutsize] = dummy
                     normflat = normflat * superflat
           except:
                print('  No superflat found for filter {}, proceeding without it'.format(filter))

           reduced=data-masterbias-darkval
           reduced=reduced/normflat/exptime*gain

           #Make a saturation Mask
           satmask = (data>60000)*1   

           #Crop cutsize pixels from each edge to avoid vignetting, preserve WCS
           reduced = reduced[cutsize:-cutsize,cutsize:-cutsize]
           satmask = satmask[cutsize:-cutsize,cutsize:-cutsize]
           header['CRPIX1'] -= cutsize
           header['CRPIX2'] -= cutsize

           #Now run fluxcal
           calpars, psfpars = flux_cal_gaia(reduced, satmask, header, filter, caldir='./calstars/')
           if np.logical_not(np.isfinite(calpars[0])):
               calpars = [0,0,0,3]
           if calpars[0] !=0:
              reduced *= calpars[0] /1e-17

           header['FCALFAC'] = calpars[0]
           header['FCALSTD'] = calpars[1]
           header['FCALSTR'] = calpars[3]
           header['ABZP'] = calpars[2]
           header['FUNIT'] = '1e-17 erg/s/cm2/A'
           header['PSFFWHM'] = psfpars[0]
           header['PSFAXRAT'] = psfpars[2]

           if addsuperflat:
              fits.writeto(reduxpath+"/"+img.replace('.fits','_reduced2.fits').split('/')[-1],reduced,header,overwrite=True)
           else:
              fits.writeto(reduxpath+"/"+img.replace('.fits','_reduced.fits').split('/')[-1],reduced,header,overwrite=True)


Reducing science images for date: 2025-10-02
Using calibration bias from date: 2025-09-25
  Using calibration flat from date: 2025-10-01


Set MJD-AVG to 60950.840603 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/NGC6946/
  Image Calibration factor in filter G: 5.88e-17 +/- 2.21e-18 (AB ZP 19.77)
  PSF FWHM: 2.53 arcsec. Axis Ratio: 1.23
  Using calibration flat from date: 2025-10-01


Set MJD-AVG to 60950.844570 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/NGC6946/
  Image Calibration factor in filter G: 5.99e-17 +/- 2.35e-18 (AB ZP 19.74)
  PSF FWHM: 2.72 arcsec. Axis Ratio: 1.18
  Using calibration flat from date: 2025-10-01


...

Reading GAIA spectra from: ./calstars/M_29/
  Image Calibration factor in filter G: 5.35e-17 +/- 1.87e-18 (AB ZP 19.87)
  PSF FWHM: 2.80 arcsec. Axis Ratio: 1.07
  Using calibration flat from date: 2025-10-30


Set MJD-AVG to 60972.731532 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/M_29/
  Image Calibration factor in filter G: 5.16e-17 +/- 2.06e-18 (AB ZP 19.91)
  PSF FWHM: 2.67 arcsec. Axis Ratio: 1.06
  Using calibration flat from date: 2025-10-30


Set MJD-AVG to 60972.744002 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/M_29/
  Image Calibration factor in filter I: 1.22e-16 +/- 2.98e-18 (AB ZP 18.00)
  PSF FWHM: 2.36 arcsec. Axis Ratio: 1.07


In [40]:
'''
#Generate superflats for each filter, does not really work

#Find observations in each filter
filters = ['R','G','I','SII','OIII','Ha','Hb']

for filter in filters:
    reduximages = glob.glob(reduxpath+'/*_{}_*_reduced.fits'.format(filter))
    if len(reduximages) < 10:
        print('Not enough images found for filter {}'.format(filter))
        continue
    else:
        print('Found {} images for filter {}'.format(len(reduximages), filter))

    #Stack images to create superflat
    stack = []
    for filein in reduximages:
        hdu = fits.open(filein)
        exptime = hdu[0].header['EXPTIME']
        if exptime<200:
            continue
        data = (hdu[0].data).astype(np.float32)
        bkg = sep.Background(data)
        data_sub = data - bkg
        sources, segmap = sep.extract(data_sub, 0.5, minarea=10, err=bkg.globalrms, segmentation_map=True)
        data[segmap>0] = np.nan
        print(filein, np.shape(data))
        stack.append(data/np.nanmedian(data))

    stack = np.array(stack)
    #Compute median
    superflat = np.nanmedian(stack, axis=0)
    #Normalize superflat
    superflat /= np.nanmedian(superflat)
    #Save superflat
    fits.writeto(calibpath+'/SuperFlat_{}.fits'.format(filter), superflat, overwrite=True)
'''

"\n#Generate superflats for each filter, does not really work\n\n#Find observations in each filter\nfilters = ['R','G','I','SII','OIII','Ha','Hb']\n\nfor filter in filters:\n    reduximages = glob.glob(reduxpath+'/*_{}_*_reduced.fits'.format(filter))\n    if len(reduximages) < 10:\n        print('Not enough images found for filter {}'.format(filter))\n        continue\n    else:\n        print('Found {} images for filter {}'.format(len(reduximages), filter))\n\n    #Stack images to create superflat\n    stack = []\n    for filein in reduximages:\n        hdu = fits.open(filein)\n        exptime = hdu[0].header['EXPTIME']\n        if exptime<200:\n            continue\n        data = (hdu[0].data).astype(np.float32)\n        bkg = sep.Background(data)\n        data_sub = data - bkg\n        sources, segmap = sep.extract(data_sub, 0.5, minarea=10, err=bkg.globalrms, segmentation_map=True)\n        data[segmap>0] = np.nan\n        print(filein, np.shape(data))\n        stack.append(data/n

In [ ]:
'''#for each image, run sep, mask sources and fit sky background
import sep
import numpy as np

reduximages = glob.glob(reduxpath+'/*_reduced.fits')
diffrun=True
#print(reduximages)

for fileout in reduximages:
   if diffrun and os.path.isfile(fileout.replace('_reduced.fits','_sky_sub.fits')):
    print('Skipping file {}'.format(fileout))
    continue
   else:
    hdu = fits.open(fileout)
    data = (hdu[0].data).astype(np.float32)
    bkg = sep.Background(data)

    data_sub = data - bkg
    sources, segmap = sep.extract(data_sub, 0.5, minarea=10, err=bkg.globalrms, segmentation_map=True)
    segmap[segmap>0] = 1

    bkg = sep.Background(data, bh=128, bw=128,mask=segmap)

    sky_median = np.median(data[segmap==0])
    maskravel = (segmap==0).ravel()
    #fit a 2D polynomial to the sky background using astropy
    y, x = np.mgrid[:data.shape[0], :data.shape[1]]
    A = np.c_[x.ravel()**2, y.ravel()**2, x.ravel()*y.ravel(), x.ravel(), y.ravel(), np.ones(data.shape).ravel()]
    C, _, _, _ = np.linalg.lstsq(A[maskravel], data.ravel()[maskravel], rcond=None)
    sky_fit = (C[0]*x**2 + C[1]*y**2 + C[2]*x*y + C[3]*x + C[4]*y + C[5]).reshape(data.shape)
    data_sky_sub = data - sky_fit

    hdu[0].data = data_sky_sub
    hdu.writeto(fileout.replace('_reduced.fits','_sky_sub.fits'), overwrite=True)
'''

In [15]:
#for each image, run sep, mask sources and fit sky background
import sep
import numpy as np

reduximages = glob.glob(reduxpath+'/*PSZ*_reduced.fits')
diffrun=False
filtsize=3
#print(reduximages)

for fileout in reduximages:
   if diffrun and os.path.isfile(fileout.replace('_reduced.fits','_sky_sub.fits')):
    print('Skipping file {}'.format(fileout))
    continue
   else:
     with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        hdu = fits.open(fileout)
        data = (hdu[0].data).astype(np.float32)
        bkg = sep.Background(data)
        wcs = WCS(hdu[0].header)

        targname = os.path.basename(fileout).split('_')[3]
        #query astroquery to get the size of the target in arcmin
        from astroquery.simbad import Simbad

        result_table = Simbad.query_object(targname)

        if result_table is None:
            print('Could not find target {} in Simbad, using default size of 10 arcmin'.format(targname))
            xpos = data.shape[1] / 2
            ypos = data.shape[0] / 2
        else:
            try:
             #Find object RA and DEC
             obj_ra = result_table['ra'][0]
             obj_dec = result_table['dec'][0]
             print('Query succeeded')
             from astropy.coordinates import SkyCoord
             from astropy import units as u
             xpos, ypos = wcs.wcs_world2pix(obj_ra, obj_dec, 0)
            except:
             xpos = data.shape[1] / 2
             ypos = data.shape[0] / 2

        #Convert size from arcmin to pixels
        size_arcmin = 10
        size_pix = (size_arcmin * 60.) / 0.55 / 2

        data_sub = data - sep.Background(data, fw=filtsize, fh=filtsize)
        try:
            sources, segmap = sep.extract(data_sub, 1.0, minarea=10, err=bkg.globalrms, segmentation_map=True)
        except:
            sources, segmap = sep.extract(data_sub, 2.5, minarea=10, err=bkg.globalrms, segmentation_map=True, deblend_cont=1.0)
        segmap[segmap>0] = 1

        #Mask a circular region around the target only if targname is not M29 or M39 or M103
        if targname != 'M29' and targname != 'M39' and targname != 'M103' and targname!='PSZ2G114.79-33.71':
           y, x = np.mgrid[:data.shape[0], :data.shape[1]]
           r = np.sqrt((x - xpos)**2 + (y - ypos)**2)
           segmap[r < size_pix] = 1

        bkg = sep.Background(data, fw=filtsize, fh=filtsize, mask=segmap)

        hdu[0].data = data - bkg
        hdu.writeto(fileout.replace('_reduced.fits','_sky_sub.fits'), overwrite=True)


Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded
Query succeeded


Coadding

In [17]:
import glob
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np

# List of filters to process
filters = ['R','G','I','SII','OIII','Ha','Hb']
targname = 'PSZ2G114.79-33.71'
outsize = 4500

for filter in filters:
    # Find all sky-subtracted images for this filter
    images = glob.glob(reduxpath + f'/*{targname}_{filter}*sky_sub.fits')
    if len(images) == 0:
        print('No images found')
        continue
    else:
        print(f'Processing {len(images)} images for filter {filter}')

    #Generate an header of 5000x5000 pixels with WCS centered on the target and pixel scale of 0.55 arcsec/pixel
    # Use the first image as reference
    ref_hdu = fits.open(images[0])
    ref_data = ref_hdu[0].data
    ref_header = ref_hdu[0].header

    from astroquery.simbad import Simbad
    result_table = Simbad.query_object(targname)
    if result_table is None:
        print('Could not find target {} in Simbad, using default RA and DEC'.format(targname))
    else:
        print('Query succeeded for target {}'.format(targname))
        try:
         #Find object RA and DEC
         center_ra = result_table['ra'][0]
         center_dec = result_table['dec'][0]
         print('Target coordinates: {:7.4f} - {:7.4f} '.format(center_ra, center_dec))
         ref_header['CRVAL1'] = center_ra
         ref_header['CRVAL2'] = center_dec
         ref_header['CRPIX1'] = outsize/2.
         ref_header['CRPIX2'] = outsize/2.
         ref_header['CD1_1'] = -0.0001527778
         ref_header['CD2_2'] = 0.0001527778
         ref_header['CD1_2'] = 0
         ref_header['CD2_1'] = 0
         ref_data = np.zeros((outsize,outsize))
        except:
         print('Could not find target {} in Simbad, using default RA and DEC'.format(targname))

    ref_wcs = WCS(ref_header)
    stack = []

    for img in images:
      with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        hdu = fits.open(img)
        data = hdu[0].data
        header = hdu[0].header
        wcs = WCS(header)
        exptime = hdu[0].header['EXPTIME']
        if exptime<30:
            print(f'Skipping image {img} with exptime {exptime}s')
            continue
        # Reproject to reference WCS
        reproj_data, _ = reproject_interp((data, wcs), ref_wcs, shape_out=ref_data.shape, parallel=4)
        stack.append(reproj_data)

    # Combine (with sigma clipped mean statistics)
    print('Stacking ....')
    stack = np.array(stack)
    clip = sigma_clip(stack, axis=0, masked=False)
    combined = np.nanmean(clip, axis=0)
    combvar  = np.nanvar(clip, axis=0)

    # Save combined image and uncertainty
    hdu1 = fits.PrimaryHDU(combined, ref_header)
    hdu2 = fits.ImageHDU(combvar, ref_header)
    hduout = fits.HDUList([hdu1, hdu2])

    hduout.writeto(combinepath + f'/Combined_{targname}_{filter}.fits', overwrite=True)

Processing 60 images for filter R
Query succeeded for target PSZ2G114.79-33.71
Target coordinates:  5.1614 - 28.6946 


Set MJD-AVG to 60986.886251 from DATE-AVG'. [astropy.wcs.wcs]


Stacking ....


/var/folders/ls/6hrs6lfd0h3_n2gkfnmq_k940000gn/T/ipykernel_17720/2527257562.py:72: RuntimeWarning: Mean of empty slice
  combined = np.nanmean(clip, axis=0)
/var/folders/ls/6hrs6lfd0h3_n2gkfnmq_k940000gn/T/ipykernel_17720/2527257562.py:73: RuntimeWarning: Degrees of freedom <= 0 for slice.
  combvar  = np.nanvar(clip, axis=0)


No images found
No images found
No images found
No images found
No images found
No images found


In [ ]:
'''
#For the students: Generate a bespoke header for the combined image
import glob
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np

# List of filters to process
filters = ['R','G','I','SII','OIII','Ha','Hb']
targname = 'NGC6946'
ra_cent = 308.71804
dec_cent =  60.15367
outsize = 4500

for filter in filters:
    # Find all sky-subtracted images for this filter
    images = glob.glob(reduxpath + f'/*{targname}_{filter}*sky_sub.fits')
    if len(images) == 0:
        print('No images found')
        continue
    else:
        print(f'Processing {len(images)} images for filter {filter}')

    ref_wcs = WCS(naxis=2)
    # reference value = center coordinates (deg)
    ref_wcs.wcs.crval = [ra_cent, dec_cent]
    # reference pixel = image center (1-indexed convention handled by astropy as float)
    ref_wcs.wcs.crpix = [outsize / 2.0, outsize / 2.0]
    # set projection and units
    ref_wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    ref_wcs.wcs.cunit = ["deg", "deg"]

    # pixel scale in degrees
    scale_deg = 0.55 / 3600.0

    ref_wcs.wcs.cd = np.array([[-scale_deg, 0.0], [0.0, scale_deg]])

    stack = []

    for img in images:
        hdu = fits.open(img)
        data = hdu[0].data
        header = hdu[0].header
        wcs = WCS(header)
        exptime = hdu[0].header['EXPTIME']
        if exptime<30:
            print(f'Skipping image {img} with exptime {exptime}s')
            continue
        # Reproject to reference WCS
        reproj_data, _ = reproject_interp((data, wcs), ref_wcs, shape_out=ref_data.shape, parallel=4)
        stack.append(reproj_data)

    # Combine (with sigma clipped mean statistics)
    print('Stacking ....')
    stack = np.array(stack)
    clip = sigma_clip(stack, axis=0, masked=False)
    combined = np.nanmean(clip, axis=0)

    # Save combined image
    fits.writeto(combinepath + f'/Combined_{targname}_{filter}.fits', combined, ref_header, overwrite=True)
'''